<a href="https://colab.research.google.com/github/FTi130/parareal-alg-python/blob/master/PararealAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
try:
    import cupy as cp
except ImportError:
    # Install the appropriate CuPy version for the CUDA version in Colab
    !pip install cupy-cuda11x  # Use 'cupy-cuda12x' if CUDA version is 12.x in Colab
    import cupy as cp

In [ ]:
# Define time parameters and differential equation

T = 1.0         # Final time
N = 10          # Number of time intervals
dt = T / N      # Time step

# Differential equation: dx/dt = -kx (simple decay equation)
k = 1.0         # Decay constant

In [ ]:
def exact_solution(t, x0):
    """
    Analytical solution for comparison, x(t) = x0 * exp(-k * t)
    """
    return x0 * np.exp(-k * t)

In [ ]:
# Coarse and Fine Solvers
def coarse_solver(x0, dt):
    """Coarse solver: a simple, less accurate Euler method"""
    return x0 - k * x0 * dt  # Coarse approximation (Euler's method)

In [ ]:
def fine_solver(x0, dt):
    """
    Fine solver: a more accurate method, e.g., using a RK2 (Midpoint) method on GPU.
    """
    x = cp.array(x0)  # Transfer initial condition to GPU
    k_cp = cp.array(k)
    dt_cp = cp.array(dt)

    # Midpoint method (Runge-Kutta2)
    k1 = -k_cp * x
    k2 = -k_cp * (x + 0.5 * dt_cp * k1)
    x_new = x + dt_cp * k2

    return cp.asnumpy(x_new)  # Transfer result back to CPU

In [ ]:
# Parareal Algorithm itself
def parareal_algorithm(x0, T, N, max_iter=10, tol=1e-5):
    dt = T / N
    times = np.linspace(0, T, N + 1)

    # Step 1: Initialize coarse solution
    X_coarse = np.zeros(N + 1)  # Solution storage
    X_coarse[0] = x0
    for n in range(N):
        X_coarse[n + 1] = coarse_solver(X_coarse[n], dt)

    # Step 2: Parareal iterations
    for iteration in range(max_iter):
        print(f"Iteration {iteration+1}")

        X_fine = np.zeros(N + 1)  # Fine solution at each iteration
        X_fine[0] = x0

        # Run fine solver in parallel on each time subinterval
        corrections = np.zeros(N + 1)
        for n in range(N):
            # Predict next fine solution (fine solver correction)
            X_fine[n + 1] = fine_solver(X_coarse[n], dt)

            # Update using coarse correction
            corrections[n + 1] = X_coarse[n + 1] + (X_fine[n + 1] - coarse_solver(X_coarse[n], dt))

        # Update coarse solution
        X_coarse = corrections

        # Check convergence
        if np.linalg.norm(X_fine - X_coarse) < tol:
            print("Converged!")
            break

    return X_coarse, X_fine


In [ ]:
# Initial condition
x0 = 1.0

In [ ]:
X_coarse, X_fine = parareal_algorithm(x0, T, N)

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10


In [ ]:
# Comparison with exact solution
exact_sol = exact_solution(np.linspace(0, T, N + 1), x0)

print("\nCoarse solution:\n", X_coarse)
print("\nFine solution:\n", X_fine)
print("\nExact solution:\n", exact_sol)


Coarse solution:
 [0.         0.905      0.855225   0.770517   0.6934748  0.62412739
 0.56171465 0.50554319 0.45498887 0.40948998 0.36854098]

Fine solution:
 [1.         0.         0.819025   0.7698835  0.69346847 0.62412735
 0.56171465 0.50554319 0.45498887 0.40948998 0.36854098]

Exact solution:
 [1.         0.90483742 0.81873075 0.74081822 0.67032005 0.60653066
 0.54881164 0.4965853  0.44932896 0.40656966 0.36787944]
